## Module 2 Homework

In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

If not stated otherwise, please use the [LINK](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/02-dataframe-analysis/%5B2025%5D_Module_02_Colab_Working_with_the_data.ipynb) covered at the livestream to re-use the code snippets.

---
### Question 1: [IPO] Withdrawn IPOs by Company Type

**What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?**

From the withdrawn IPO list ([stockanalysis.com/ipos/withdrawn](https://stockanalysis.com/ipos/withdrawn/)), collect and process the data to find out which company type saw the most withdrawn IPO value.

#### Steps:
1. Use `pandas.read_html()` with the URL above to load the IPO withdrawal table into a DataFrame.
   *It is a similar process to Code Snippet 1 discussed at the livestream.*    You should get **99 entries**.
2. Create a new column called `Company Class`, categorizing company names based on patterns like:
   - “Acquisition Corp” or “Acquisition Corporation” → `Acq.Corp`
   - “Inc” or “Incorporated” → `Inc`
   - “Group” → `Group`
   - “Holdings” → `Holdings`
   - “Ltd” or “Limited” → `Ltd`
   - Others → `Other`

  * Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.
  * Order: Please follow the listed order of classes and assign the first matched value (e.g., for 'shenni holdings limited', you assign the 'Limited' class).

3. Define a new field `Avg. price` by parsing the `Price Range` field (create a function and apply it to the `Price Range` column). Examples:
   - '$8.00-$10.00' → `9.0`  
   - '$5.00' → `5.0`  
   - '-' → `None`
4. Convert `Shares Offered` to numeric, clean missing or invalid values.
5. Create a new column:  
   `Withdrawn Value = Shares Offered * Avg Price` (**71 non-null values**)
6. Group by `Company Class` and calculate total withdrawn value.
7. **Answer**: Which class had the highest **total** value of withdrawals?
---

In [1]:
import pandas as pd
import numpy as np
import requests
import re
import yfinance as yf
import time

pd.set_option('display.max_columns', None)

In [2]:
# 1. Use `pandas.read_html()` with the URL above to load the IPO withdrawal table into a DataFrame.
#    *It is a similar process to Code Snippet 1 discussed at the livestream.*    You should get **99 entries**.
url = 'https://stockanalysis.com/ipos/withdrawn/'

# Add header to handle html error 403
# https://stackoverflow.com/questions/43590153/http-error-403-forbidden-when-reading-html
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

response = requests.get(url, headers=header)
df_list = pd.read_html(response.text)
df = df_list[0]
df.info()

/tmp/ipython-input-2-1398987536.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(response.text)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [3]:
df.head()

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-


In [4]:
def get_company_class(company_name):
  if 'acquisition corp' in company_name:
    return 'Acq.Corp'
  if 'inc' in company_name:
    return 'Inc'
  if 'group' in company_name:
    return 'Group'
  if ('ltd' in company_name) or ('limited' in company_name):
    return 'Ltd'
  if 'holdings' in company_name:
    return 'Holdings'
  return 'Other'


In [5]:
# 2. Create a new column called `Company Class`, categorizing company names based on patterns like:
#    - “Acquisition Corp” or “Acquisition Corporation” → `Acq.Corp`
#    - “Inc” or “Incorporated” → `Inc`
#    - “Group” → `Group`
#    - “Holdings” → `Holdings`
#    - “Ltd” or “Limited” → `Ltd`
#    - Others → `Other`

#   * Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.
df['Company Name'] = df['Company Name'].str.lower()
df['Company Class'] = df['Company Name'].apply(get_company_class)
df.head()

,Symbol,Company Name,Price Range,Shares Offered,Company Class
0,ODTX,"odyssey therapeutics, inc.",-,-,Inc
1,UNFL,"unifoil holdings, inc.",$3.00 - $4.00,2000000,Inc
2,AURN,"aurion biotech, inc.",-,-,Inc
3,ROTR,"phi group, inc.",-,-,Inc
4,ONE,one power company,-,-,Other


In [6]:
def get_avg_price(price_range):
  if price_range == '-':
    return None
  else:
    prices = re.findall(r'\d+\.\d{2}', price_range)
    if len(prices) == 1:
      return float(prices[0])
    else:
      avg_price = (float(prices[0]) + float(prices[1])) / 2
    return avg_price

In [7]:
# 3. Define a new field `Avg. price` by parsing the `Price Range` field (create a function and apply it to the `Price Range` column). Examples:
#    - '$8.00-$10.00' → `9.0`
#    - '$5.00' → `5.0`
#    - '-' → `None`
df['Avg. price'] = df['Price Range'].apply(get_avg_price)

# 4. Convert `Shares Offered` to numeric, clean missing or invalid values.
df['Shares Offered'] = pd.to_numeric(df['Shares Offered'], errors='coerce')

df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71 entries, 1 to 99
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Symbol          71 non-null     object 
 1   Company Name    71 non-null     object 
 2   Price Range     71 non-null     object 
 3   Shares Offered  71 non-null     float64
 4   Company Class   71 non-null     object 
 5   Avg. price      71 non-null     float64
dtypes: float64(2), object(4)
memory usage: 3.9+ KB


In [8]:
# 5. Create a new column:
#    `Withdrawn Value = Shares Offered * Avg Price` (**71 non-null values**)
df['Withdrawn Value'] = df['Shares Offered'] * df['Avg. price']

# 6. Group by `Company Class` and calculate total withdrawn value.
df.groupby('Company Class')[['Withdrawn Value']].sum().sort_values(by='Withdrawn Value', ascending=False)

# 7. **Answer**: Which class had the highest **total** value of withdrawals?

/tmp/ipython-input-8-143821429.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Withdrawn Value'] = df['Shares Offered'] * df['Avg. price']


,Withdrawn Value
Company Class,
Acq.Corp,4.021000e+09
Inc,2.257164e+09
Other,7.679200e+08
Ltd,5.497346e+08
Holdings,7.500000e+07
Group,3.378750e+07


### Question 2:   [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)


**What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?**

The goal is to replicate the large-scale `yfinance` OHLCV data download and perform basic financial calculations on IPO stocks.


#### Steps:

1. Using the same approach as in Question 1, download the IPOs in 2024 from:  
   [https://stockanalysis.com/ipos/2024/](https://stockanalysis.com/ipos/2024/)  
   Filter to keep only those IPOs **before 1 June 2024** (first 5 months of 2024).  
   ➤ You should have **75 tickers**.

2.  Use **Code Snippet 7** to download daily stock data for those tickers (via `yfinance`).  
   Make sure you understand how `growth_1d` ... `growth_365d`, and volatility columns are defined.  
   Define a new column `growth_252d` representing growth after **252 trading days** (~1 year), in addition to any other growth periods you already track.


3. Calculate the Sharpe ratio assuming a risk-free rate of **4.5%**:

   ```python
   stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']
4. Filter the DataFrame to keep data only for the trading day:  
   **‘2025-06-06’**

   Compute descriptive statistics (e.g., `.describe()`) for these columns:  
   - `growth_252d`  
   - `Sharpe`

   You should observe:  
   - `growth_252d` is defined for **71 out of 75 stocks** (some IPOs are too recent or data starts later).  
   - Median `growth_252d` is approximately **0.75** (indicating a 25% decline), while mean is about **1.15**, showing a bias towards high-growth companies pushing the average up.

5. **Answer:**  
   - What is the **median Sharpe ratio** for these 71 stocks?  
   - Note: Positive `Sharpe` means growth exceeding the risk-free rate of 4.5%.  
   - [Additional] Do you observe the **same top 10 companies** when sorting by `growth_252d` versus sorting by `Sharpe`?


In [9]:
# 1. Using the same approach as in Question 1, download the IPOs in 2024 from:
#    [https://stockanalysis.com/ipos/2024/](https://stockanalysis.com/ipos/2024/)

resp = requests.get('https://stockanalysis.com/ipos/2024/', headers=header)
df_ipo = pd.read_html(resp.text)[0]
print(df_ipo.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      225 non-null    object
 1   Symbol        225 non-null    object
 2   Company Name  225 non-null    object
 3   IPO Price     225 non-null    object
 4   Current       225 non-null    object
 5   Return        225 non-null    object
dtypes: object(6)
memory usage: 10.7+ KB
None


/tmp/ipython-input-9-3688517787.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_ipo = pd.read_html(resp.text)[0]


In [10]:
#    Filter to keep only those IPOs **before 1 June 2024** (first 5 months of 2024).
#    ➤ You should have **75 tickers**.
df_ipo['IPO Date'] = pd.to_datetime(df_ipo['IPO Date'])
df_ipo = df_ipo[(df_ipo['IPO Date'] > '2024-01-01') & (df_ipo['IPO Date'] < '2024-06-01')]
df_ipo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77 entries, 148 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      77 non-null     datetime64[ns]
 1   Symbol        77 non-null     object        
 2   Company Name  77 non-null     object        
 3   IPO Price     77 non-null     object        
 4   Current       77 non-null     object        
 5   Return        77 non-null     object        
dtypes: datetime64[ns](1), object(5)
memory usage: 4.2+ KB


In [11]:
df_ipo.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
148,2024-05-31,NAKA,"Kindly MD, Inc.",-,$14.00,-
149,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,$17.00,$36.35,114.65%
150,2024-05-17,HDL,Super Hi International Holding Ltd.,$19.56,$19.37,-0.31%
151,2024-05-17,RFAI,RF Acquisition Corp II,$10.00,$10.53,5.30%
152,2024-05-15,JDZG,JIADE Limited,$4.00,$0.28,-93.00%


In [13]:
# ignore tickers without IPO price, there are two such tickers.
df_ipo[df_ipo['IPO Price']=='-']

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
148,2024-05-31,NAKA,"Kindly MD, Inc.",-,$14.00,-
174,2024-04-17,SUPX,SuperX AI Technology Limited,-,$9.80,-


In [14]:
# keep only those tickers with IPO price
df_ipo = df_ipo[df_ipo['IPO Price']!='-']
df_ipo.shape

(75, 6)

In [15]:
# 2.  Use **Code Snippet 7** to download daily stock data for those tickers (via `yfinance`).
#    Make sure you understand how `growth_1d` ... `growth_365d`, and volatility columns are defined.
#    Define a new column `growth_252d` representing growth after **252 trading days** (~1 year), in addition to any other growth periods you already track.
tickers = list(df_ipo['Symbol'].values)
stocks_df = pd.DataFrame({'A' : []})

for t in tickers:
  # Work with stock prices
  ticker_obj = yf.Ticker(t)

  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = t
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365,252]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)

  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

In [16]:
# 3. Calculate the Sharpe ratio assuming a risk-free rate of **4.5%**:

#    ```python
#    stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']
stocks_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,Weekday,Date,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_365d,growth_252d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future,Sharpe
0,23.000000,24.270000,22.139999,23.799999,3335800,0.0,0.0,BOW,2024,5,3,2024-05-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.092017,NaN,NaN,0,0.089496,NaN,1,NaN
1,24.260000,26.150000,23.980000,25.700001,990500,0.0,0.0,BOW,2024,5,4,2024-05-24,1.079832,NaN,NaN,NaN,NaN,NaN,NaN,0.998054,NaN,NaN,0,0.084436,NaN,0,NaN
2,25.850000,26.879999,25.075001,26.480000,555100,0.0,0.0,BOW,2024,5,1,2024-05-28,1.030350,NaN,NaN,NaN,NaN,NaN,NaN,1.001133,NaN,NaN,0,0.068165,NaN,1,NaN
3,26.440001,26.490000,25.500999,26.290001,302700,0.0,0.0,BOW,2024,5,2,2024-05-29,0.992825,1.104622,NaN,NaN,NaN,NaN,NaN,0.987828,NaN,NaN,0,0.037619,NaN,0,NaN
4,27.209999,27.209999,25.500000,26.139999,200900,0.0,0.0,BOW,2024,5,3,2024-05-30,0.994294,1.017121,NaN,NaN,NaN,NaN,NaN,1.037490,NaN,NaN,0,0.065417,NaN,1,NaN


In [17]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])
stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23241 entries, 0 to 23240
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Open                           23241 non-null  float64       
 1   High                           23241 non-null  float64       
 2   Low                            23241 non-null  float64       
 3   Close                          23241 non-null  float64       
 4   Volume                         23241 non-null  int64         
 5   Dividends                      23241 non-null  float64       
 6   Stock Splits                   23241 non-null  float64       
 7   Ticker                         23241 non-null  object        
 8   Year                           23241 non-null  int32         
 9   Month                          23241 non-null  int32         
 10  Weekday                        23241 non-null  int32         
 11  Date           

In [18]:
# 4. Filter the DataFrame to keep data only for the trading day:
#    **‘2025-06-06’**

#    Compute descriptive statistics (e.g., `.describe()`) for these columns:
#    - `growth_252d`
#    - `Sharpe`

#    You should observe:
#    - `growth_252d` is defined for **71 out of 75 stocks** (some IPOs are too recent or data starts later).
#    - Median `growth_252d` is approximately **0.75** (indicating a 25% decline), while mean is about **1.15**,
# showing a bias towards high-growth companies pushing the average up.
df_66 = stocks_df.loc[stocks_df['Date'] == '2025-06-06', ['growth_252d', 'Sharpe']]
print(f'Total num. of stocks: {df_66.shape[0]}')
df_66.describe()

Total num. of stocks: 75


,growth_252d,Sharpe
count,71.000000,71.000000
mean,1.152898,0.288285
std,1.406017,0.519028
min,0.024970,-0.079677
25%,0.293422,0.041215
50%,0.758065,0.083768
75%,1.362736,0.311507
max,8.097413,2.835668


In [19]:
# 5. **Answer:**
#    - What is the **median Sharpe ratio** for these 71 stocks?
#    - Note: Positive `Sharpe` means growth exceeding the risk-free rate of 4.5%.
#    - [Additional] Do you observe the **same top 10 companies** when sorting by `growth_252d` versus sorting by `Sharpe`?
df_66['Sharpe'].describe()

,Sharpe
count,71.000000
mean,0.288285
std,0.519028
min,-0.079677
25%,0.041215
50%,0.083768
75%,0.311507
max,2.835668


### Question 3: [IPO] ‘Fixed Months Holding Strategy’

**What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?**  
(*Assume you buy at the close of the first trading day and sell after a fixed number of trading days.*)


---

#### Goal:
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

---

#### Steps:

1. **Start from the existing DataFrame** from Question 2 (75 tickers from IPOs in the first 5 months of 2024).  

   Add **12 new columns**:  
   `future_growth_1m`, `future_growth_2m`, ..., `future_growth_12m`  
   *(Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)*  
   This logic is similar to `historyPrices['growth_future_30d']` from **Code Snippet 7**, but extended to longer timeframes.

2. **Determine the first trading day** (`min_date`) for each ticker.  
   This is the earliest date in the data for each stock.

3. **Join the data**:  
   Perform an **inner join** between the `min_date` DataFrame and the future growth data on both `ticker` and `date`.  
   ➤ You should end up with **75 records** (one per IPO) with all 12 `future_growth_...` fields populated.

4. **Compute descriptive statistics** for the resulting DataFrame:  
   Use `.describe()` or similar to analyze each of the 12 columns:  
   - `future_growth_1m`  
   - `future_growth_2m`  
   - ...  
   - `future_growth_12m`  

5. **Determine the best holding period**:  
   - Find the number of months **(1 to 12)** where the **average (mean)** future growth is **maximal**.  
   - This optimal month shows an uplift of **>1%** compared to all others.  
   - Still, the average return remains **less than 1** (i.e., expected return is less than doubling your investment).


In [28]:
stocks_growth = stocks_df.copy()

# Add 12 new columns:
# future_growth_1m, future_growth_2m, ..., future_growth_12m
# (Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)
# This logic is similar to historyPrices['growth_future_30d'] from Code Snippet 7, but extended to longer timeframes.
for i in range(1,13):
  stocks_growth[f'future_growth_{i}m'] = stocks_growth['Close'].shift(i * -21) / stocks_growth['Close']

In [29]:
# Determine the first trading day (min_date) for each ticker.
# Perform an inner join between the min_date DataFrame and the future growth data on both ticker and date.
# ➤ You should end up with 75 records (one per IPO) with all 12 future_growth_... fields populated.
stocks_growth = stocks_growth.loc[stocks_growth.groupby(['Ticker'])['Date'].idxmin()]
stocks_growth.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75 entries, 16289 to 3489
Data columns (total 39 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Open                           75 non-null     float64       
 1   High                           75 non-null     float64       
 2   Low                            75 non-null     float64       
 3   Close                          75 non-null     float64       
 4   Volume                         75 non-null     int64         
 5   Dividends                      75 non-null     float64       
 6   Stock Splits                   75 non-null     float64       
 7   Ticker                         75 non-null     object        
 8   Year                           75 non-null     int32         
 9   Month                          75 non-null     int32         
 10  Weekday                        75 non-null     int32         
 11  Date                

In [34]:
# 4. **Compute descriptive statistics** for the resulting DataFrame:
#    Use `.describe()` or similar to analyze each of the 12 columns:
#    - `future_growth_1m`
#    - `future_growth_2m`
#    - ...
#    - `future_growth_12m`
for i in range(1,13):
  mean = stocks_growth[f'future_growth_{i}m'].mean()
  median = stocks_growth[f'future_growth_{i}m'].median()
  print(f'{i}m : {mean: .3f}, {median: .3f}')
# 5. **Determine the best holding period**:
#    - Find the number of months **(1 to 12)** where the **average (mean)** future growth is **maximal**.
#    - This optimal month shows an uplift of **>1%** compared to all others.
#    - Still, the average return remains **less than 1** (i.e., expected return is less than doubling your investment).

1m :  0.927,  0.977
2m :  0.941,  1.000
3m :  0.834,  0.928
4m :  0.825,  0.909
5m :  0.804,  0.821
6m :  0.864,  0.802
7m :  0.847,  0.845
8m :  0.833,  0.812
9m :  0.882,  0.823
10m :  12.584,  0.778
11m :  12.583,  0.720
12m :  12.603,  0.711
